Data Preparation

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
# Load data
george_images = [f for f in os.listdir('./george_test_task/george') if f.endswith('.jpg')]
non_george_images = [f for f in os.listdir('./george_test_task/no_george') if f.endswith('.jpg')]

Image Preprocessing

In [3]:
def load_preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.
    return image

Creating Datasets

In [4]:
# Load and preprocess images
george_images_data = np.array([load_preprocess_image(os.path.join('./george_test_task/george', img)) for img in george_images])
non_george_images_data = np.array([load_preprocess_image(os.path.join('./george_test_task/no_george', img)) for img in non_george_images])

# Create labels
george_labels = np.ones((len(george_images), 1))
non_george_labels = np.zeros((len(non_george_images), 1))

# Combine data and labels
X = np.concatenate((george_images_data, non_george_images_data))
y = np.concatenate((george_labels, non_george_labels))

Creating Model

In [5]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten

# Load pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model Training

In [6]:

from sklearn.model_selection import train_test_split
# Combine data and labels
X = np.concatenate((george_images_data, non_george_images_data))
y = np.concatenate((george_labels, non_george_labels))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1643s 11s/step - accuracy: 0.6848 - loss: 1.1640 - val_accuracy: 0.8263 - val_loss: 0.4046
Epoch 2/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1338s 9s/step - accuracy: 0.8989 - loss: 0.2656 - val_accuracy: 0.8640 - val_loss: 0.3401
Epoch 3/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1257s 9s/step - accuracy: 0.9369 - loss: 0.1803 - val_accuracy: 0.8711 - val_loss: 0.3350
Epoch 4/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1247s 9s/step - accuracy: 0.9672 - loss: 0.1176 - val_accuracy: 0.8614 - val_loss: 0.3719
Epoch 5/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1385s 10s/step - accuracy: 0.9855 - loss: 0.0734 - val_accuracy: 0.8675 - val_loss: 0.3654
Epoch 6/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1347s 9s/step - accuracy: 0.9901 - loss: 0.0507 - val_accuracy: 0.8491 - val_loss: 0.4660
Epoch 7/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1260s 9s/step - accuracy: 0.9896 - loss: 0.0435 - val_accuracy: 0.8456 - val_loss: 0.5092
Epoch 8/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 1286s 9s/step - accuracy: 0.9927 - loss: 0.0288 

Evolution of Model

In [7]:
from sklearn.metrics import accuracy_score
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the test accuracy
print(f'Test accuracy: {test_accuracy}')

# Make predictions on the test data
y_pred = model.predict(X_test)

# Convert predicted probabilities to binary labels
y_pred = np.round(y_pred)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.8561403751373291
36/36 ━━━━━━━━━━━━━━━━━━━━ 338s 9s/step
Test accuracy: 0.856140350877193


Prediction

In [8]:
def predict_image(model, image_path):
    image = load_preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    if prediction > 0.5:
        print ('St. George present')
    else:
        print ('St. George absent')

Present or Absent

In [10]:
# Load an image
image_path = 'george_test_task/george/0a67ef257ce20427b5a55b94bcac2521.jpg'

# Make a prediction on the image
predict_image(model, image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step
St. George present


In [11]:
# Load an image
image_path = 'george_test_task/no_george/0a869d67deaaa70385fae7f70b92a557.jpg'

# Make a prediction on the image
predict_image(model, image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
St. George absent
